In [1]:
using JuMP
#using GLPKMathProgInterface
using CPLEX

# Generación de columnas

La generación de columnas no difiere mucho entre problemas. La idea básica siempre es:

(1) Tenemos un modelo que puede dividirse en un problema tipo "set cover" o "set packing" y otro que genera los sets del modelo de set cover o set packing.

Un ejemplo típico es el cutting stock. El modelo master resultante es:

$\mbox{[MIN]} \sum_{j\in \mathcal{J}} x_j$ 

$\sum_{j\in \mathcal{J}} a_{ij} x_j \geq d_i \qquad \forall i\in P$

$x_j \in \mathbb{Z}_{\geq0}\qquad \forall j\in \mathcal{J}$

donde las variables indican el número de veces que se repite el patrón en rollos diferentes y se intenta minimizart el número de rollos utilizados para cumplir con la demanda $d_i$ de los $P$ productos.

El problema de pricing usa los costes reducidos de las variables del dual para calcular el mejor rollo:

$ \mbox{[MIN]} 1 - \sum_{i \in P} {\pi_i y_i}$

sujeto a:

$ \sum_{i \in P} w_i y_{i} \leq W $

$ y_i \in \mathbb{Z}_{\geq0}\qquad \forall j\in \mathcal{J} $

Que corresponde a un modelo de knapsack tradicional.

Nota: Consultar http://www4.ncsu.edu/~kksivara/ma505/handouts/gilmore-gomory1.pdf para detalles de lo que estamos haciendo.

In [3]:
function instanciaAleatoria(n::Int64)
    w = rand(1:1000, n)
    d=rand(1:50, n)
    W=sum(w[1:n])/5
    if W<maximum(w)
        W=maximum(w)
    end
    return W,d,w
end

instanciaAleatoria (generic function with 1 method)

In [7]:
function solveCG(n,W,d,w,solver=CplexSolver(),solverPricing=CplexSolver())
    EPS=0.000001
    model = Model(solver = solver)
    @variable(model, x[1:n]>=0)
    @objective(model,Min, sum(x[i] for i in 1:n) )
    @constraint(model, demand[i=1:n], x[i] >= d[i])
    
    pricing = Model(solver = solverPricing)
    @variable(pricing, y[1:n]>=0,Int)
    @constraint(pricing,sum(w[i]*y[i] for i in 1:n)<=W)
    numVariables=n
    p=zeros(Float64,n)
    while true
        status = solve(model)
        print("solucion: ",status,"\t",getobjectivevalue(model),"\t")
        p=getdual(demand)
        #el pricing sólo requiere un cambio de la función objetivo
        @objective(pricing,Max,sum(p[i]*y[i] for i in 1:n))
        status = solve(pricing)
        println("\tpricing:\t",getobjectivevalue(pricing))
        if getobjectivevalue(pricing)<(1.0+EPS)
            break
        else
            #new column
            solvec=Float64[]
            for i in 1:n
                push!(solvec,getvalue(y)[i])
            end
            @variable(model, xNew>=0, objective = 1, inconstraints=demand, coefficients =solvec)
            numVariables += 1
            #setname(xNew, string("x[",numVariables,"]")) 
        end
    end    
    println("\nend")
    print(model)
end

solveCG (generic function with 3 methods)

In [8]:
n=10
W,d,w=instanciaAleatoria(100)
solveCG(n,W,d,w,CplexSolver(CPX_PARAM_SCRIND=0),CplexSolver(CPX_PARAM_SCRIND=0))

solucion: Optimal	329.0		pricing:	49.0
solucion: Optimal	298.63265306122446		pricing:	25.020408163265305
solucion: Optimal	287.10285714285715		pricing:	25.0
solucion: Optimal	252.54285714285712		pricing:	25.0
solucion: Optimal	206.46285714285713		pricing:	21.0
solucion: Optimal	166.4628571428571		pricing:	19.0
solucion: Optimal	158.88390977443606		pricing:	18.047619047619047
solucion: Optimal	114.37068226120859		pricing:	16.020408163265305
solucion: Optimal	78.69721287345348		pricing:	12.0
solucion: Optimal	60.36387954012013		pricing:	11.059591836734695
solucion: Optimal	17.381987146799176		pricing:	1.0165270318653779
solucion: Optimal	17.373723630866486		pricing:	1.3450340136054423
solucion: Optimal	17.368290885040885		pricing:	1.0133333333333334
solucion: Optimal	17.340731170245878		pricing:	1.0119047619047619
solucion: Optimal	17.328425194136425		pricing:	1.0095238095238095
solucion: Optimal	17.30827946793691		pricing:	1.007875150060024
solucion: Optimal	17.295425096914684		pricing:

## Método como una heurística

Una vez resuelto, podríamos aprovechar el método para usarlo como heurística, para ello tendremos que cambiar las variables para que pasen a ser enteras.

In [9]:
function solveCGHeur(n,W,d,w,solver=CplexSolver(),solverPricing=CplexSolver())
    EPS=0.000001
    model = Model(solver = solver)
    newColumns=Variable[]
    @variable(model, x[1:n]>=0)
    @objective(model,Min, sum(x[i] for i in 1:n) )
    @constraint(model, demand[i=1:n], x[i] >= d[i])
    
    pricing = Model(solver = solverPricing)
    @variable(pricing, y[1:n]>=0,Int)
    @constraint(pricing,sum(w[i]*y[i] for i in 1:n)<=W)
    numVariables=n
    p=zeros(Float64,n)
    while true
        status = solve(model)
        print("solucion: ",status,"\t",getobjectivevalue(model),"\t")
        p=getdual(demand)
        #el pricing sólo requiere un cambio de la función objetivo
        @objective(pricing,Max,sum(p[i]*y[i] for i in 1:n))
        status = solve(pricing)
        println("\tpricing:\t",getobjectivevalue(pricing))
        if getobjectivevalue(pricing)<(1.0+EPS)
            break
        else
            #new column
            solvec=Float64[]
            for i in 1:n
                push!(solvec,getvalue(y)[i])
            end
            @variable(model, xNew>=0, objective = 1, inconstraints=demand, coefficients =solvec)
            numVariables += 1
            setname(xNew, string("x[",numVariables,"]")) 
            push!(newColumns, xNew)            
        end
    end    
    println("\nend Fase 1 con valor:",getobjectivevalue(model))
    for i=1:length(x)
        setcategory(x[i], :Int)
    end
    for i=1:length(newColumns)
        setcategory(newColumns[i], :Int)
    end
    function infocallback(m)
        println("exploredNodes: ",MathProgBase.cbgetexplorednodes(m)," obj: ",MathProgBase.cbgetobj(m), " bestBound: ",MathProgBase.cbgetbestbound(m))        
    end
    addinfocallback(model, infocallback, when = :Intermediate)
    status = solve(model)
    println("\nend Fase 2 con valor:",getobjectivevalue(model))
end

solveCGHeur (generic function with 3 methods)

In [ ]:
solveCGHeur(n,W,d,w,CplexSolver(CPX_PARAM_SCRIND=0),CplexSolver(CPX_PARAM_SCRIND=0))

solucion: Optimal	329.0		pricing:	49.0
solucion: Optimal	298.63265306122446		pricing:	25.020408163265305
solucion: Optimal	287.10285714285715		pricing:	25.0
solucion: Optimal	252.54285714285712		pricing:	25.0
solucion: Optimal	206.46285714285713		pricing:	21.0
solucion: Optimal	166.4628571428571		pricing:	19.0
solucion: Optimal	158.88390977443606		pricing:	18.047619047619047
solucion: Optimal	114.37068226120859		pricing:	16.020408163265305
solucion: Optimal	78.69721287345348		pricing:	12.0
solucion: Optimal	60.36387954012013		pricing:	11.059591836734695
solucion: Optimal	17.381987146799176		pricing:	1.0165270318653779
solucion: Optimal	17.373723630866486		pricing:	1.0150340136054423
solucion: Optimal	17.368290885040885		pricing:	1.9533333333333334
solucion: Optimal	17.340731170245878		pricing:	1.0119047619047619
solucion: Optimal	17.328425194136425		pricing:	1.0095238095238095
solucion: Optimal	17.30827946793691		pricing:	1.007875150060024
solucion: Optimal	17.295425096914684		pricing: